#Libraries


In [25]:
import sagemaker
import ipykernel
from sklearn.model_selection import train_test_split
import boto3
import pandas as pd
#import matplotlib.pyplot as plt

sm_boto3 = boto3.client("sagemaker")
sess = sagemaker.Session()
region = sess.boto_session.region_name
bucket = 'bucketprojectsagemaker'

In [26]:
df = pd.read_csv("./datasets/datosequivalentes.csv")
df.head(10)

,ID_DETALLE,NOMCIENTIFICO,ALTURATOTAL,VOLUMENCOMER,DENSIDAD,ESTADO_COPA,ESTADO_FUSTE,ESTADO_RAIZ,EST_RAIZ_GEN,NIVEL_RIESGO,ESTADO_FITO,TRATAMIENTO
0,1612750498351,Ficus elastica,13.0,0.000,Denso,Bueno,Bueno,No apreciable,Bueno,Moderada,Regular,Poda Estructural
1,1639177795281,Eugenia myrtifolia,8.0,0.000,Denso,Bueno,Bueno,No apreciable,Bueno,Moderada,Regular,Poda Aclareo
2,1631544761286,Eucalyptus globulus,7.0,0.018,Muy Ralo,Regular,Regular,No apreciable,Bueno,Severa,Regular,Tala
3,1660188850013,Phylantus sp,3.8,0.000,Denso,Bueno,Bueno,No apreciable,Bueno,Ninguna,Regular,Conservacion
4,1664837550230,Macleania rupestris,8.1,0.000,Medio,Bueno,Bueno,No apreciable,Bueno,Ninguna,NaN,Conservacion
5,1639676317090,Fraxinus chinensis,16.0,0.000,Medio,Regular,Regular,Raices Descubiertas,Regular,Moderada,Regular,Tratamiento Integral
6,1638851799172,Ficus soatensis,11.0,0.000,Denso,Bueno,Bueno,Raices estranguladoras,Bueno,Ninguna,Regular,Tratamiento Integral
7,12405,Lafoensia acuminata,14.0,0.000,Denso,Malo,Regular,No apreciable,Bueno,Severa,NaN,Poda Estructural
8,1429897167695,Pittosporum undulatum,10.0,0.000,Denso,Regular,Regular,No apreciable,Regular,Moderada,Regular,Tala
9,1416261107216,Sambucus nigra,8.0,0.000,Denso,Regular,Bueno,No apreciable,Regular,Ninguna,Regular,Poda Estructural


In [27]:
df.columns

Index(['ID_DETALLE', 'NOMCIENTIFICO', 'ALTURATOTAL', 'VOLUMENCOMER',
       'DENSIDAD', 'ESTADO_COPA', 'ESTADO_FUSTE', 'ESTADO_RAIZ',
       'EST_RAIZ_GEN', 'NIVEL_RIESGO', 'ESTADO_FITO', 'TRATAMIENTO'],
      dtype='object')

In [28]:
df.shape

(10100, 12)

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10100 entries, 0 to 10099
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID_DETALLE     10100 non-null  int64  
 1   NOMCIENTIFICO  10100 non-null  object 
 2   ALTURATOTAL    10100 non-null  float64
 3   VOLUMENCOMER   10100 non-null  float64
 4   DENSIDAD       10100 non-null  object 
 5   ESTADO_COPA    10100 non-null  object 
 6   ESTADO_FUSTE   10100 non-null  object 
 7   ESTADO_RAIZ    10100 non-null  object 
 8   EST_RAIZ_GEN   10100 non-null  object 
 9   NIVEL_RIESGO   10100 non-null  object 
 10  ESTADO_FITO    3294 non-null   object 
 11  TRATAMIENTO    10100 non-null  object 
dtypes: float64(2), int64(1), object(9)
memory usage: 947.0+ KB


In [30]:
df= df[['ID_DETALLE','TRATAMIENTO', 'ALTURATOTAL', 'DENSIDAD', 'ESTADO_COPA', 'ESTADO_FUSTE', 'EST_RAIZ_GEN', 'NIVEL_RIESGO', 'ESTADO_FITO']].dropna()

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3294 entries, 0 to 10095
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ID_DETALLE    3294 non-null   int64  
 1   TRATAMIENTO   3294 non-null   object 
 2   ALTURATOTAL   3294 non-null   float64
 3   DENSIDAD      3294 non-null   object 
 4   ESTADO_COPA   3294 non-null   object 
 5   ESTADO_FUSTE  3294 non-null   object 
 6   EST_RAIZ_GEN  3294 non-null   object 
 7   NIVEL_RIESGO  3294 non-null   object 
 8   ESTADO_FITO   3294 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 257.3+ KB


In [32]:
densidad_type = {'Denso':0,'Medio':1, 'Muy Ralo':2, 'Ralo':3}
estado_types = {'Bueno':0,'Regular':1, 'Malo':2, 'Suprimido':4}
riesgo_types = {'Moderada':0,'Ninguna':1, 'Severa':2}

df['DENSIDAD'] = df['DENSIDAD'].map(densidad_type).astype(int) 
df['ESTADO_COPA'] = df['ESTADO_COPA'].map(estado_types).astype(int)  
df['ESTADO_FUSTE'] = df['ESTADO_FUSTE'].map(estado_types).astype(int) 
df['EST_RAIZ_GEN'] = df['EST_RAIZ_GEN'].map(estado_types).astype(int)  
df['ESTADO_FITO'] = df['ESTADO_FITO'].map(estado_types).astype(int)  
df['NIVEL_RIESGO'] = df['NIVEL_RIESGO'].map(riesgo_types).astype(int) 

tratamientos_types = {'Tala':0, 'Conservacion':1, 'Traslado':2, 'Poda Formacion':3, 'Poda Control de Altura':4, 'Poda Realce':5, 
                    'Poda Aclareo':6, 'Poda Equilibrio':7, 'Poda Mejoramiento':8, 'Poda Estructural':9, 'Poda Saneamiento':10, 
                      'Poda Radicular':11, 'Tratamiento Integral':12, 'Tratamiento Especial':13}
 
df['TRATAMIENTO'] = df['TRATAMIENTO'].map(tratamientos_types).astype(int)  

In [33]:
df.describe()

,ID_DETALLE,TRATAMIENTO,ALTURATOTAL,DENSIDAD,ESTADO_COPA,ESTADO_FUSTE,EST_RAIZ_GEN,NIVEL_RIESGO,ESTADO_FITO
count,3.294000e+03,3294.000000,3294.000000,3294.000000,3294.000000,3294.000000,3294.000000,3294.000000,3294.0
mean,1.574504e+12,3.081360,6.756670,0.870370,0.546145,0.551305,0.341226,0.389192,1.0
std,8.138612e+10,4.560162,5.268987,0.819525,0.695412,0.725407,0.572808,0.507178,0.0
min,1.415131e+12,0.000000,0.300000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
25%,1.550410e+12,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
50%,1.550410e+12,1.000000,5.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.0
75%,1.641873e+12,5.000000,9.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0
max,1.662351e+12,12.000000,36.000000,3.000000,2.000000,4.000000,2.000000,2.000000,1.0


In [34]:
df.head(10)

,ID_DETALLE,TRATAMIENTO,ALTURATOTAL,DENSIDAD,ESTADO_COPA,ESTADO_FUSTE,EST_RAIZ_GEN,NIVEL_RIESGO,ESTADO_FITO
0,1612750498351,9,13.0,0,0,0,0,0,1
1,1639177795281,6,8.0,0,0,0,0,0,1
2,1631544761286,0,7.0,2,1,1,0,2,1
3,1660188850013,1,3.8,0,0,0,0,1,1
5,1639676317090,12,16.0,1,1,1,1,0,1
6,1638851799172,12,11.0,0,0,0,0,1,1
8,1429897167695,0,10.0,0,1,1,1,0,1
9,1416261107216,9,8.0,0,1,0,1,1,1
11,1424357756573,0,4.0,0,2,2,0,0,1
12,1415130817207,0,20.0,1,1,2,0,2,1


In [35]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X = df[['ALTURATOTAL', 'DENSIDAD', 'ESTADO_COPA', 'ESTADO_FUSTE', 'EST_RAIZ_GEN', 'NIVEL_RIESGO', 'ESTADO_FITO']].values
y = df[['TRATAMIENTO']].values

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=0)

In [37]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(2799, 7)
(495, 7)
(2799, 1)
(495, 1)


In [ ]:
features = list({'ALTURATOTAL', 'DENSIDAD', 'ESTADO_COPA', 'ESTADO_FUSTE', 
                 'EST_RAIZ_GEN', 'NIVEL_RIESGO', 'ESTADO_FITO'})
label = 'TRATAMIENTO'
trainX = pd.DataFrame(X_train)
trainX[label] = y_train

testX = pd.DataFrame(X_test)
testX[label] = y_test

In [39]:
print(trainX.shape)
print(testX.shape)

(2799, 8)
(495, 8)


In [40]:
trainX.head()

,0,1,2,3,4,5,6,TRATAMIENTO
0,9.0,3.0,1.0,1.0,0.0,0.0,1.0,0
1,4.0,1.0,0.0,0.0,0.0,0.0,1.0,0
2,2.5,1.0,1.0,0.0,1.0,1.0,1.0,1
3,6.0,1.0,0.0,0.0,0.0,1.0,1.0,1
4,7.5,1.0,0.0,0.0,0.0,1.0,1.0,9


In [41]:
trainX.isnull().sum()

0              0
1              0
2              0
3              0
4              0
5              0
6              0
TRATAMIENTO    0
dtype: int64

In [42]:
trainX.to_csv("train-V-1.csv", index=False)
testX.to_csv("test-V-1.csv", index=False)

In [43]:
#send data to S3
sk_prefix = "sagemaker/tratamientos/sklearncontainer"
trainpath = sess.upload_data(
    path = "train-V-1.csv", bucket=bucket, key_prefix=sk_prefix
)

testpath = sess.upload_data(
    path = "test-V-1.csv", bucket=bucket, key_prefix=sk_prefix
)

In [44]:
%%writefile script.py

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score
import sklearn
import joblib
import boto3
import pathlib
from io import StringIO
import argparse
import joblib
import os
import numpy as np
import pandas as pd

def model_fn(model_dir):
  clf = joblib.load(os.path.join(model_dir, "model.joblib"))
  return clf

if __name__ == "__main__":
  print("[INFO] Extracting arguments")
  parser = argparse.ArgumentParser()

  # hyperparameters sent by the client are passed as command-line arguments to
  parser.add_argument("--n_estimators", type=int, default=100)
  parser.add_argument("--random_state", type=int, default=0)

  # Data, model, and output directories
  parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
  parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
  parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))
  parser.add_argument("--train-file", type=str, default="train-V-1.csv")
  parser.add_argument("--test-file", type=str, default="test-V-1.csv")

  args, _ = parser.parse_known_args()

  # Imprime las versiones de scikit-learn y joblib
  print("Scikit-learn Version:", sklearn.__version__)
  print("Joblib Version:", joblib.__version__)

  # Indica que se están leyendo los datos
  print("[INFO] Reading data")
  print()

  # Carga los datos de entrenamiento y prueba desde los archivos CSV especificados
  train_df = pd.read_csv(os.path.join(args.train, args.train_file))
  test_df = pd.read_csv(os.path.join(args.test, args.test_file))

  # Obtiene las características (features) de los datos de entrenamiento
  features = list(train_df.columns)

  # Extrae la última columna (asumiendo que es la etiqueta) y la asigna a la variable 'label'
  label = features.pop(-1)

  # Indica que se están construyendo los conjuntos de entrenamiento y prueba
  print("Building training and testing datasets")
  print()

  X_train = train_df[features]
  X_test = test_df[features]
  y_train = train_df[label]
  y_test = test_df[label]

  print('Column order:')
  print(features)
  print()

  print("Label column is: ", label)
  print()
  print("---- SHAPE OF TRAINING DATA (85%) ----")
  print(X_train.shape)
  print(y_train.shape)
  print()

  print("---- SHAPE OF TESTING DATA (15%) ----")
  print(X_test.shape)
  print(y_test.shape)
  print()

  print("---- Training RandomForest Model ----")
  model = RandomForestClassifier(n_estimators=100, 
  max_depth=2, random_state=0)
  model.fit(X_train, y_train)
  print()

  model_path = os.path.join(args.model_dir, "model.joblib")
  joblib.dump(model, model_path)
  print("Model persisted at " + model_path)
  print()

  y_pred_test = model.predict(X_test)
  test_acc = accuracy_score(y_test, y_pred_test)
  test_rep = classification_report(y_test, y_pred_test)

  print()
  print("---- METRICS RESULTS FOR TESTING DATA ----")
  print()
  print("Total Rows are: ", X_test.shape[0])
  print("[TESTING] Model Accuracy is: ", test_acc)
  print("[TESTING] Testing Report:")
  print(test_rep)




Overwriting script.py


In [45]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"

sklearn_estimator = SKLearn(
    entry_point="script.py",
    role="arn:aws:iam::445567096458:role/service-role/SageMaker-Datos",
    instance_count=1,
    instance_type="ml.m5.large",
    framework_version=FRAMEWORK_VERSION,
    base_job_name="RF-custom-sklearn",
    hyperparameters={
        "n_estimators": 100,
        "random_state": 0,
    },
    use_spot_instances = True,
    max_wait = 7200,
    max_run = 3600
)

c:\Users\JP711\OneDrive - ESCUELA COLOMBIANA DE INGENIERIA JULIO GARAVITO\Documentos\MAESTRIA\2024-2\AYGO\Project\myenv\lib\site-packages\boto3\compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [46]:
# launch training job, with asynchronous call
sklearn_estimator.fit({"train": trainpath, "test": testpath}, wait=True)
# sklearn_estimator.fit({"train": datapath}, wait=True)


Using provided s3_resource


INFO:sagemaker:Creating training-job with name: RF-custom-sklearn-2024-12-06-23-08-20-082


2024-12-06 23:08:22 Starting - Starting the training job...
2024-12-06 23:08:38 Starting - Preparing the instances for training...
2024-12-06 23:09:14 Downloading - Downloading input data...
2024-12-06 23:09:45 Downloading - Downloading the training image...
2024-12-06 23:10:20 Training - Training image download completed. Training in progress..2024-12-06 23:10:24,469 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-12-06 23:10:24,473 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-12-06 23:10:24,516 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-12-06 23:10:24,669 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-12-06 23:10:24,682 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-12-06 23:10:24,696 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-12-06 23:

In [47]:
sklearn_estimator.latest_training_job.wait(logs="None")
artifact = sm_boto3.describe_training_job(
    TrainingJobName=sklearn_estimator.latest_training_job.name
)["ModelArtifacts"]["S3ModelArtifacts"]
print("Model artifact persisted at " + artifact)


2024-12-06 23:10:43 Starting - Preparing the instances for training
2024-12-06 23:10:43 Downloading - Downloading the training image
2024-12-06 23:10:43 Training - Training image download completed. Training in progress.
2024-12-06 23:10:43 Uploading - Uploading generated training model
2024-12-06 23:10:43 Completed - Training job completed
Model artifact persisted at s3://sagemaker-us-east-1-445567096458/RF-custom-sklearn-2024-12-06-23-08-20-082/output/model.tar.gz


In [49]:
from sagemaker.sklearn.model import SKLearnModel
from time import gmtime, strftime

model_name = "Custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model = SKLearnModel(
    name=model_name,
    model_data=artifact,
    role="arn:aws:iam::445567096458:role/service-role/SageMaker-Datos",
    entry_point="script.py",
    framework_version=FRAMEWORK_VERSION,
)
model

In [50]:
endpoint_name = "Custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName={}".format(endpoint_name))

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
    endpoint_name=endpoint_name
)
endpoint_name

EndpointName=Custom-sklearn-model-2024-12-06-23-25-21


INFO:sagemaker:Creating model with name: Custom-sklearn-model-2024-12-06-23-23-33
INFO:sagemaker:Creating endpoint-config with name Custom-sklearn-model-2024-12-06-23-25-21
INFO:sagemaker:Creating endpoint with name Custom-sklearn-model-2024-12-06-23-25-21


------!

'Custom-sklearn-model-2024-12-06-23-25-21'